# Lựa chọn đặc trưng and LASSO

Trong notebook này, chúng ta sẽ sử dụng LASSO để lựa chọn đặc trưng, xây dựng solver đã triển khai trước cho LASSO (trong sklearn). Cụ thể, chúng ta sẽ:  
* Chạy LASSO với các L1 penalty khác nhau.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định với ràng buộc bổ sung về kích thước tập con. 

Trong bài tập tiếp theo, chúng ta sẽ tạo LASSO solver sử dụng coordinate descent.

## Thư viện

In [102]:
import sklearn
import pandas
import numpy as np

import warnings

warnings.filterwarnings('ignore')

## Load tập dữ liệu doanh số bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [103]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)

## Tạo các đặc trung mới

Như ở lab 2 (*Lab-2.ipynb*), chúng ta sẽ xem xét các đặc trưng có các biến đổi đầu vào.

In [104]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# Trong tập dữ liệu, 'floors' được xác định là type string, 
# nên chúng ta sẽ chuyển chúng thành float trước khi tạo đặc trưng mới. 
full_data['floors'] = full_data['floors'].astype(float) 
full_data['floors_square'] = full_data['floors'] ** 2

* Bình phương bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn 1) và nhiều phòng ngủ (chẳng hạn 4) vì 1^2 = 1 còn 4^2 = 16. Do đó, biến này sẽ ảnh hưởng lớn tới các ngôi nhà có nhiều phòng ngủ.
* Mặt khác, căn bậc hai của sqft_living sẽ giảm phân tách giữa nhà lớn và nhà nhỏ. Chủ ngôi nhà cũng sẽ không vui hơn nếu nhà rộng gấp đôi.

# Tìm hiểu trọng số hồi quy với L1 penalty

Hãy khớp mô hình với tất cả đặc trưng hiện có cộng với các đặc trưng vừa tạo.

In [105]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Áp dụng L1 penalty cần thêm tham số (`alpha=l1_penalty`) cho mô hình  `Lasso` của Sklearn. (Các công cụ khác cũng phân tách các triển khai của LASSO). Khá giống Hồi quy Ridge/L2, các đặc trưng cũng cần được co giãn để đảm bảo đồng đều ở giữa.

In [106]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Tìm các đặc trưng có trọng số khác 0.

In [109]:
# Bạn có biết numpy cũng có boolean selector tích hợp sẵn?
def non_zero(model):
    coefs = model.coef_
    index_selected_feature = [x for x,v in enumerate(coefs) if abs(v) !=0]
    return index_selected_feature

Lưu ý rằng phần lớn trọng số được đặt thành 0. Vì vậy, bằng cách đặt L1 penalty đủ lớn, chúng ta có thể thực hiện lựa chọn tập con.

***QUIZ***:
Theo list các trọng số này, những đặc trưng nào đã được chọn?

In [111]:
index_selected_feature = non_zero(model)
selected_feature = [all_features[x] for x in index_selected_feature]
print(index_selected_feature)
print(selected_feature)

[3, 9, 10, 12, 15]
['sqft_living', 'waterfront', 'view', 'grade', 'yr_built']


# Lựa chọn L1 penalty

Để tìm một L1 penalty tốt, chúng ta sẽ khám phá nhiều giá trị sử dụng tập kiểm định. Hãy chia dữ liệu thành tập huấn luyện, tập kiểm định và tập kiểm tra:
* Chia dữ liệu bán hàng thành 2 tập: tập huấn luyện và tập kiểm tra (9/1)
* Chia tiếp tập huấn luyện thành 2 tập: tập huấn luyện và kiểm định (5/5)

Hãy dùng seed = 1 để có cùng kết quả!

In [112]:
from sklearn.model_selection import train_test_split
# Cookie cho những ai cần copy cell 
train_and_validation, test = train_test_split(full_data, train_size=0.9, random_state=1)
train, valid = train_test_split(train_and_validation, train_size=0.5, random_state=1)

Tiếp theo, chúng ta sẽ viết một vòng lặp như sau: 
* Với `l1_penalty` trong phạm vi 21 bước giữa [1, 10^9] (sử dụng `np.logspace(0, 9, num=21)`.)
    * Khớp mô hình hồi quy với `l1_penalty` trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Tính RSS trên dữ liệu KIỂM ĐỊNH (sử dụng `.predict()`) cho `l1_penalty`
* Báo lại `l1_penalty` nào cho RSS thấp nhất trong dữ liệu KIỂM ĐỊNH.

In [117]:
# Mức độ trợ giúp: Bình thường.
l1_penalty_values = np.logspace(0, 9, num=21)
def train_model(data_train, data_test, in_features, out_feature, l1_penalty_values, max_nonzero = None, list_l1_penalty = []):
    best_l1_penalty = None
    min_RSS = np.Inf
    best_model = None
    
    for l1_penalty in l1_penalty_values:
        X_train = scaler.fit_transform(train[all_features].values)
        y_train = train['price'].values
        model = Lasso(alpha=l1_penalty).fit(X_train, y_train)
        
        if max_nonzero is not None:
            non_zeros = len(non_zero(model))
            if non_zeros == max_nonzeros:
                list_l1_penalty.append(l1_penalty)
        
        X_valid = scaler.fit_transform(valid[all_features].values)
        y_valid = valid['price'].values

        prediction = model.predict(X_valid)
        RSS = ((y_valid - prediction)**2).sum()
        
        if RSS < min_RSS:
            min_RSS = RSS
            best_l1_penalty = l1_penalty
            best_model = model
    non_zeros = non_zero(best_model)  
    return best_l1_penalty, min_RSS, non_zeros, list_l1_penalty

best_l1_penalty, min_RSS, non_zeros, list_l1_penalty = train_model(train, test, all_features,'price', l1_penalty_values)

In [120]:
print(f'min_RSS: {min_RSS} at best l1_penalty: {best_l1_penalty} and no.nonzeros: {len(non_zeros)}')
print(list_l1_penalty)

min_RSS: 414402087272830.1 at best l1_penalty: 501.18723362727246 and no.nonzeros: 17
[]


***QUIZ:*** Giá trị tốt nhất cho `l1_penalty` là bao nhiêu?<br>


In [94]:
# Qua quan sát hay tính toán?

***QUIZ***
Với giá trị L1 penalty này, chúng ta có bao nhiêu trọng số khác 0?

In [95]:
# Thú vị phải không?

# Limit the number of nonzero weights Giới hạn số trọng số khác 0

Sẽ ra sao nếu chúng ta muốn giới hạn, 5 đặc trưng chẳng hạn? Điều này quan trọng nếu chúng ta muốn suy ra "quy tắc ngón tay cái" --- mô hình có thể diễn giải chỉ với một vài đặc trưng.

Trong phần này, chúng ta sẽ triển khai quy trình đơn giản gồm 2 giai đoạn :
1. Thăm dò phạm vi lớn các giá trị `l1_penalty` để tìm vùng các giá trị `l1_penalty` hẹp hơn mà mô hình chắc chắn sẽ có số lượng trọng số khác 0 mong muốn.
2. Thăm dò tiếp vùng hẹp đã thấy để tìm gái trị tốt cho `l1_penalty` đạt được độ thưa thớt mong muốn. Ở đây chúng ta sử dụng tập kiểm định để chọn giá trị tốt nhất cho `l1_penalty`. 

In [121]:
max_nonzeros = 5

## Khám phá phạm vi giá trị lớn hơn để tìm phạm vi hẹp với độ thưa thớt mong muốn

Hãy xác định một loạt các `l1_penalty_values` có thể:

In [123]:
l1_penalty_values = np.logspace(3, 5, num=21)

Giờ hãy triển khi vòng lặp tìm kiếm trong không gian có các giá trị `l1_penalty` có thể:

* Với `l1_penalty` trong `np.logspace(3, 5, num=21)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Trích xuất trọng số của mô hình và đếm số trọng số khác 0. Lưu con số này vào một list. 
        * Gợi ý: `model.coef_` cho các tham số/hệ số đã tìm thấy (intercept) ở dạng mảng numpy. Sau đó có thể dùng array\[condition\] cho list các giá trị truyền điều kiện, hoặc chỉ dùng `np.count_nonzero()` có sẵn.

In [124]:
best_l1_penalty, min_RSS, non_zeros, list_l1_penalty = train_model(train, test, all_features,'price', l1_penalty_values, max_nonzeros)

In [126]:
print(list_l1_penalty)

[25118.864315095823, 31622.776601683792, 39810.71705534969, 50118.72336272725]


Trong phạm vi lớn này, chúng ta có thể tìm 2 đầu phạm vi hẹp mong muốn của `l1_penalty`. Ở một đầu, các giá trị `l1_penalty` có quá ít giá trị khác 0, còn đầu kia lại có quá nhiều giá trị khác 0.

Hãy tìm:
* `l1_penalty` nhỏ nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty nhỏ hơn giá trị này chắc chắn sẽ có rất nhiều trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_min` (sẽ sử dụng nó sau).
* `l1_penalty` lớn nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty lớn hơn giá trị này chắc chắn sẽ có rất ít trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_max` (sẽ sử dụng nó sau).


*Gợi ý: có nhiều cách để thực hiện, chẳng hạn:*
* Lập trình trong vòng lặp trên.
* Tạo một list với số lượng khác 0 cho từng giá trị `l1_penalty` và kiểm tra nó để tìm ranh giới thích hợp.

In [127]:
l1_penalty_min = list_l1_penalty[0]
l1_penalty_max = list_l1_penalty[-1]

***QUIZ.*** Chúng ta tìm thấy các giá trị nào lần lượt cho `l1_penalty_min` và `l1_penalty_max`? 

## Khám phá phạm vi nhỏ các giá trị để tìm giải pháp với đúng số lượng các số khác 0 có RSS trong tập kiểm định nhỏ nhất

Chúng ta sẽ khám phá vùng hẹp các giá trị `l1_penalty` đã tìm thấy:

In [128]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print(l1_penalty_values)

[25118.8643151  26434.64637023 27750.42842537 29066.21048051
 30381.99253565 31697.77459079 33013.55664593 34329.33870107
 35645.1207562  36960.90281134 38276.68486648 39592.46692162
 40908.24897676 42224.0310319  43539.81308703 44855.59514217
 46171.37719731 47487.15925245 48802.94130759 50118.72336273]


* Với `l1_penalty` trong `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty`.
    * Đo lường RSS của mô hình đã tìm hiểu trong tập KIỂM ĐỊNH.

Tìm mô hình có RSS nhỏ nhất trong tập KIỂM ĐỊNH và độ thưa thớt *bằng*  `max_nonzeros`.

In [129]:
# Xem quiz bên dưới
best_l1_penalty, min_RSS, non_zeros, list_l1_penalty = train_model(train, test, all_features,'price', l1_penalty_values)
print(f'min_RSS: {min_RSS} at l1_penalty: {best_l1_penalty}')
print(non_zeros)

min_RSS: 491275148416788.25 at l1_penalty: 25118.864315095823
[3, 9, 10, 12, 15]


***QUIZ***
1. Giá trị của `l1_penalty` trong phạm vi hẹp hơn có RSS thấp nhất trong tập KIỂM ĐỊNH và độ thưa *bằng* `max_nonzeros` là?
2. Các đặc trung nào trong mô hình này có các hệ số khác 0?

In [131]:
selected_feature = [all_features[x] for x in non_zeros]
print(selected_feature)

['sqft_living', 'waterfront', 'view', 'grade', 'yr_built']


==========================End==========================